In [1]:
# plot the radial distribution histogram
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
halos_grpNow = [8]
sim_num = 329
latestsnapshot = 864
latesthaloid = 8
data_folder=f"/home/takeichi/MAP/Results/{sim_num}/{latestsnapshot:06d}{latesthaloid}"
csv_path=os.path.join(data_folder, f"tracked_star_particles{latestsnapshot:06d}{latesthaloid}.csv")
import pynbody as pb
import numpy as np
import pandas as pd
import glob
import os
import h5py
import tables
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import imageio
from natsort import natsorted
import importlib.util
import sys
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D
import ast
import matplotlib.patches as patches
os.chdir('/home/takeichi/MAP/halo_trace')
import halo_trace as ht
from halo_trace import tracingCopy1 as tracing
file_path = '/home/takeichi/MAP/Juan_startrace/methods/src/jstreams/star_traceCopy.py'
module_name = 'star_trace'

spec = importlib.util.spec_from_file_location(module_name, file_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

file_path1 = '/home/takeichi/MAP/Code_Yuma_2025/Code/rad_dist.py'
module_name1 = 'raddist'

spec = importlib.util.spec_from_file_location(module_name1, file_path1)
raddist = importlib.util.module_from_spec(spec)
spec.loader.exec_module(raddist)


pb.config['halo-class-priority'] = ['HaloNumberCatalogue', 'AHFCatalogue',
  'AmigaGrpCatalogue',
  'VelociraptorCatalogue',
  'SubFindHDFHaloCatalogue',
  'RockstarCatalogue', 
  'SubfindCatalogue',
  'NewAdaptaHOPCatalogue',
  'NewAdaptaHOPCatalogueFullyLongInts',
  'AdaptaHOPCatalogue',
  'HOPCatalogue',
  'Gadget4SubfindHDFCatalogue',
  'ArepoSubfindHDFCatalogue',
  'TNGSubfindHDFCatalogue']


In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pynbody as pb
import matplotlib.cm as cm
import matplotlib.colors as mcolors

infopath = "/home/takeichi/MAP/Code_Yuma_2025/Code/Datafiles/Data100_new_edited_Z.csv"
try:
    info_df = pd.read_csv(infopath)
except Exception as e:
    print(f"Fail to read csv file. Detail: {e}")
snapshots_to_analyze = [
    '004096', '004032', '003936', '003840', '003744', '003648', '003606', 
    '003552', '003456', '003360', '003264', '003195', '003168', '003072', 
    '002976', '002880', '002784', '002688', '002592', '002554', '002496', 
    '002400', '002304', '002208', '002112', '002088', '002016', '001920', 
    '001824', '001740', '001728', '001632', '001536', '001475', '001440', 
    '001344', '001269', '001248', '001152', '001106', '001056', '000974', 
    '000960', '000864', '000776', '000768', '000672', '000637', '000576', 
    '000480', '000456', '000384', '000347', '000288', '000275', '000225', 
    '000192', '000188', '000139', '000107', '000096', '000071'
]
snapshots_to_analyze = [
    '004096'
]
sim_num = 329
base_data_folder = f"/home/takeichi/MAP/Results/{sim_num}"
output_dir = os.path.join(base_data_folder, "r_dist_4_panel_plots")
os.makedirs(output_dir, exist_ok=True)


halo_ids = [
    "0030722", "0032649", "00048038", "00048010", "0004809", "00126914",
    "0013443", "00147516", "00067210", "0007764", "0008648", "00038428",
    "00038418", "0005763", "00063713", "0006729"
]
t_infall_map = {}
infall_mass_map = {}
if info_df is not None:
    for item in halo_ids:
        snapshot_at_infall = int(item[:6])
        halo_id_at_infall = int(item[6:])
        ID=f"{sim_num}{snapshot_at_infall:04d}{halo_id_at_infall}"
        print(ID)
        selected = info_df[info_df['ID']==int(ID)]
        if selected.empty:
            print("We cannnot find data")
        else:
            tinform = selected["time_infall"].iloc[0]
            t_infall_map[item] = tinform
            infall_mass = selected["infall_mass"].iloc[0]
            infall_mass_map[item] = infall_mass



if t_infall_map:
    min_t_infall = min(t_infall_map.values())
    max_t_infall = max(t_infall_map.values())
    cmap = plt.get_cmap('Spectral')
    norm = mcolors.Normalize(vmin=min_t_infall, vmax=max_t_infall)

if infall_mass_map:
    min_infall_mass = min(infall_mass_map.values())
    max_infall_mass = max(infall_mass_map.values())
    cmap1 = plt.get_cmap('Spectral')
    norm1 = mcolors.Normalize(vmin=min_infall_mass, vmax=max_infall_mass)
    
# --- Definition of function ---
def sim_base(simulation_num):
    """Give back the path to the data"""
    simbase = f'/data/Sims/h{simulation_num}.cosmo50PLK.3072g/h{simulation_num}.cosmo50PLK.3072gst5HbwK1BH/snapshots_200crit_h{simulation_num}/'
    tracebase = f'/data/Sims/h{simulation_num}.cosmo50PLK.3072g/h{simulation_num}.cosmo50PLK.3072gst5HbwK1BH/'
    return simbase, tracebase

def COM_main_halo(simulation_num, snapshot, mainhaloid_df):
    """Calculate the COM of the main halo"""
    simbase, _ = sim_base(simulation_num)
    try:
        snapshot_num = int(snapshot)
        halodata = pb.load(f"{simbase}h{simulation_num}.cosmo50PLK.3072gst5HbwK1BH.{snapshot_num:06d}")
    except Exception as e:
        print(f"pynbody file cannot be loaded for snapshot {snapshot}, {e}")
        return None, None, None
        
    boxcen = halodata.properties['boxsize'].in_units('kpc a') / 2
    
    if snapshot == 4096:
        # if snaoshot is 4096, we can use halo id 1
        h = halodata.halos(halo_numbers='v1')
        if len(h) > 1:
            main_halo_props = h[1].properties
        else:
            print(f"Error: Main halo not found in snapshot {snapshot}")
            return None, None, None
    else:
        # if not, use trace file
        try:
            main_halo_id_at_snap = mainhaloid_df[f"{snapshot:06d}"].iloc[0]
            h = halodata.halos(halo_numbers='v1')
            main_halo_props = h[main_halo_id_at_snap].properties
        except (KeyError, IndexError) as e:
            print(f"Could not find main halo ID for snapshot {snapshot}: {e}")
            return None, None, None

    x_org = main_halo_props['Xc']
    y_org = main_halo_props['Yc']
    z_org = main_halo_props['Zc']
    
    # changing units
    a = halodata.properties['a']
    h_param = halodata.properties['h']
    x = (x_org / h_param - boxcen) * a
    y = (y_org / h_param - boxcen) * a
    z = (z_org / h_param - boxcen) * a
    
    return x, y, z

# --- Process Data ---

# get the trace file of the main halo
mainhalo_trace_path = f"/data/Sims/h{sim_num}.cosmo50PLK.3072g/h{sim_num}.cosmo50PLK.3072gst5HbwK1BH/h{sim_num}.cosmo50PLK.3072gst5HbwK1BH.004096/h{sim_num}.cosmo50PLK.3072gst5HbwK1BH.004096.trace_back3_1.hdf5"
try:
    mainhaloid_df = pd.read_hdf(mainhalo_trace_path, index_col=0)
except FileNotFoundError:
    print(f"Main halo trace file not found at: {mainhalo_trace_path}")
    mainhaloid_df = None

# --- Loop Start ---
for snapshot_str in snapshots_to_analyze:
    SNAPSHOT_TO_ANALYZE = int(snapshot_str)
    
    all_r_arrays = []
    all_mass_arrays = []
    labels = []

    print(f"\n--- Starting analysis for Snapshot: {SNAPSHOT_TO_ANALYZE} ---")


    colors_for_plot=[]
    colors_for_plot1=[]
    linestyles = []

    survive_data_folder="/home/takeichi/MAP/Code_Yuma_2025/Code/Datafiles"
    survive_csv_path=os.path.join(survive_data_folder, "halo_star_properties_tidy_optimized.csv")
    main_csv_path = os.path.join(survive_data_folder, "main_halo_com.csv")
    try:
        survive_df=pd.read_csv(survive_csv_path)
    except Exception as e:
        print(f"File loading error, {e}")
    try:
        main_csv_df= pd.read_csv(main_csv_path, dtype={"snapshot":str})
    except Exception as e:
        print(f"File loading error, {e}")
    a = f"{SNAPSHOT_TO_ANALYZE:06d}"
    main_csv_snap=main_csv_df[main_csv_df["snapshot"]==a]
    x_center=main_csv_snap["0"].iloc[0]
    y_center=main_csv_snap["1"].iloc[0]
    z_center=main_csv_snap["2"].iloc[0]
    print(x_center,y_center,z_center)
    
    for halo_id in survive_df["halo_id"].unique():
        sub_survive = survive_df[(survive_df["halo_id"] == halo_id)&(survive_df["snapshot"]==SNAPSHOT_TO_ANALYZE)]
        x = sub_survive["particle_x"]
        x = x.to_numpy()
        y = sub_survive["particle_y"]
        y = y.to_numpy()
        z = sub_survive["particle_z"]
        z = z.to_numpy()
        mass = sub_survive["particle_mass"]
        mass = mass.to_numpy()

        
        r = np.sqrt((x-x_center)**2+(y-y_center)**2+(z-z_center)**2)
        
        all_r_arrays.append(r)
        all_mass_arrays.append(mass)
        labels.append(f"ID {halo_id} Snap 4096")
        colors_for_plot1.append("purple")
        colors_for_plot.append("purple")
        linestyles.append("-")

    
    for item in halo_ids:
        snapshot_at_infall = int(item[:6])
        halo_id_at_infall = int(item[6:])
        print(f"Processing data for progenitor halo_id: {halo_id_at_infall} (from snapshot {snapshot_at_infall})")
        
        data_folder = os.path.join(base_data_folder, item)
        csv_path = os.path.join(data_folder, f"tracked_star_particles{item}.csv")
        
        try:
            df = pd.read_csv(csv_path)
        except FileNotFoundError:
            print(f"--> File not found, skipping: {csv_path}")
            continue
        except Exception as e:
            print(f"--> File loading error: {e}")
            continue

        sub = df[df["snapshot"] == SNAPSHOT_TO_ANALYZE].copy()

        if sub.empty:
            # This can happen a lot
            # print(f"--> No data for snapshot {SNAPSHOT_TO_ANALYZE} in this file, skipping.")
            continue

        # Calculate the radius
        r = np.sqrt((sub['x'] - x_center)**2 + (sub['y'] - y_center)**2 + (sub['z'] - z_center)**2)
        
        all_r_arrays.append(r.values)
        all_mass_arrays.append(sub['mass'].values)
        labels.append(f"ID {halo_id_at_infall} (Snap {snapshot_at_infall})")
        if cmap and norm and item in t_infall_map:
            colors_for_plot.append(cmap(norm(t_infall_map[item])))

        if cmap1 and norm1 and item in infall_mass_map:
            colors_for_plot1.append(cmap1(norm1(infall_mass_map[item])))
        linestyles.append("--")
    # --- Make plot ---
    if all_r_arrays:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 32), sharex=True)
        fig.suptitle(f"Radial Distribution of Disrupted Stars in Snapshot {SNAPSHOT_TO_ANALYZE}", fontsize=20, y=0.95)

        all_r_flat = np.concatenate(all_r_arrays)
        all_mass_flat = np.concatenate(all_mass_arrays)
        
        valid_mask = all_r_flat > 0
        if not np.any(valid_mask):
            print("No valid star distances > 0 found. Cannot create plot.")
            plt.close(fig)
            continue
        width = 0.5
        #r_min = np.log10(np.min(all_r_flat[valid_mask]))
        r_max = np.max(all_r_flat[valid_mask])
        bins = np.arange(0, r_max+width, width)
        bin_centers = (bins[:-1] + bins[1:]) / 2

        total_mass_in_bins = np.zeros(len(bin_centers))
        all_cumulative_masses = []
        individual_mass_in_bins = []

        for r_array, mass_array, label, color, linestyle in zip(all_r_arrays, all_mass_arrays, labels, colors_for_plot1, linestyles):
            print(label)
            mass_in_bins, _ = np.histogram(r_array, bins=bins, weights=mass_array)
            total_mass_in_bins += mass_in_bins
            cumulative_mass = np.cumsum(mass_in_bins)
            all_cumulative_masses.append(cumulative_mass)
            individual_mass_in_bins.append(mass_in_bins)

            ax1.plot(bin_centers, cumulative_mass, label=label, color = color, linestyle=linestyle)

            shell_volumes = (4/3) * np.pi * (bins[1:]**3 - bins[:-1]**3)
            density = np.divide(mass_in_bins, shell_volumes, out=np.zeros_like(mass_in_bins, dtype=float), where=shell_volumes!=0)
            r2_density = bin_centers**2 * density
            ax2.plot(bin_centers, r2_density, label=label, color = color, linestyle= linestyle)
        
        total_cumulative_mass = np.cumsum(total_mass_in_bins)
        ax1.plot(bin_centers, total_cumulative_mass, color='black', linewidth=3, linestyle='--', label='Total Accreted')
        
        total_density = np.divide(total_mass_in_bins, shell_volumes, out=np.zeros_like(total_mass_in_bins, dtype=float), where=shell_volumes!=0)
        total_r2_density = bin_centers**2 * total_density
        ax2.plot(bin_centers, total_r2_density, color='black', linewidth=3, label='Total Accreted')

        for cumulative_mass, label, color, linestyle in zip(all_cumulative_masses, labels, colors_for_plot1, linestyles):
            fraction = np.divide(cumulative_mass, total_cumulative_mass, out=np.zeros_like(cumulative_mass, dtype=float), where=total_cumulative_mass!=0)
            ax3.plot(bin_centers, fraction, label=label, color = color, linestyle=linestyle)

        for individual_mass, label, color, linestyle in zip(individual_mass_in_bins, labels, colors_for_plot1, linestyles):
            fraction1 = np.divide(individual_mass, total_mass_in_bins, out=np.zeros_like(individual_mass, dtype=float), where=total_mass_in_bins!=0)
            ax4.plot(bin_centers, fraction1, label=label, color = color, linestyle=linestyle)

        ax1.set_yscale('log')
        ax1.set_ylabel(r"Cumulative Stellar Mass [$M_{\odot}$]", fontsize=14)
        ax1.legend(loc='lower right', title="Progenitor Halo", fontsize='small')
        ax1.grid(True, which="both", ls="--", alpha=0.5)

        ax2.set_yscale('log')
        ax2.set_ylabel(r"$r^2 \rho_{\star}$ [$M_{\odot} / \mathrm{kpc}$]", fontsize=14)
        ax2.legend(loc='upper left', title="Progenitor Halo", fontsize='small')
        ax2.grid(True, which="both", ls="--", alpha=0.5)

        ax3.set_ylabel("Fraction of Mass inside the sphere", fontsize=14)
        ax3.set_ylim(0, 1.05)
        ax3.legend(loc='upper left', title="Progenitor Halo", fontsize='small')
        ax3.grid(True, which="both", ls="--", alpha=0.5)

        ax4.set_ylabel("Fraction of Mass at each shell", fontsize=14)
        ax4.set_ylim(0, 1.05)
        ax4.legend(loc='upper left', title="Progenitor Halo", fontsize='small')
        ax4.grid(True, which="both", ls="--", alpha=0.5)

        
        plt.xlabel("Distance from the COM of mainhalo (kpc)", fontsize=14)
        for ax in [ax1, ax2, ax3, ax4]:
            ax.set_xscale('log')
            ax.set_xlim(left=bin_centers[0]*0.9, right=bin_centers[-1]*1.1)
        if cmap1 and norm1:
            fig.subplots_adjust(right=0.85)
            cbar_ax = fig.add_axes([0.88, 0.15, 0.03, 0.7])
            sm = plt.cm.ScalarMappable(cmap=cmap1, norm=norm1)
            sm.set_array([])
            cbar = fig.colorbar(sm, cax=cbar_ax)
            cbar.set_label('Infall Mass[Msun]')

        filename = f"r_dist_4-panel_snapshot_infallmass_{SNAPSHOT_TO_ANALYZE:06d}.png"
        output_path = os.path.join(output_dir, filename)
        plt.savefig(output_path, bbox_inches='tight')
        plt.close(fig)
        print(f"--> 4-panel plot saved successfully to: {output_path}")

    else:
        print(f"--> No data found to plot for snapshot {SNAPSHOT_TO_ANALYZE}. Plot was not created.")

print("\n--- All snapshots processed. ---")


32930722
32932649
329048038
329048010
32904809
329126914
32913443
329147516
329067210
32907764
32908648
329038428
329038418
32905763
329063713
32906729

--- Starting analysis for Snapshot: 4096 ---
1877.44745057552 -4441.044233960943 1478.1246825739918
Processing data for progenitor halo_id: 2 (from snapshot 3072)
Processing data for progenitor halo_id: 9 (from snapshot 3264)
Processing data for progenitor halo_id: 38 (from snapshot 480)
Processing data for progenitor halo_id: 10 (from snapshot 480)
Processing data for progenitor halo_id: 9 (from snapshot 480)
Processing data for progenitor halo_id: 14 (from snapshot 1269)
Processing data for progenitor halo_id: 3 (from snapshot 1344)
Processing data for progenitor halo_id: 16 (from snapshot 1475)
Processing data for progenitor halo_id: 10 (from snapshot 672)
Processing data for progenitor halo_id: 4 (from snapshot 776)
Processing data for progenitor halo_id: 8 (from snapshot 864)
Processing data for progenitor halo_id: 28 (from snapsh

In [20]:

survive_data_folder="/home/takeichi/MAP/Code_Yuma_2025/Code/Datafiles"
survive_csv_path=os.path.join(survive_data_folder, "halo_star_properties_tidy_optimized.csv")
try:
    survive_df=pd.read_csv(survive_csv_path)
except Exception as e:
    print(f"File loading error, {e}")

In [28]:
sub_survive = survive_df[(survive_df["halo_id"] == 7)&(survive_df["snapshot"]==347)]
x = sub_survive["particle_x"]
x = x.to_numpy()


54.90304955225104


In [18]:
mass = sub_survive["mass"]
mass_list_as_string = mass.iloc[0]
mass_stripped_string = mass_list_as_string.strip("[]")
#mass_stripped_string = mass_stripped_string.replace("...", " ")
mass = [float(num.strip()) for num in mass_stripped_string.split(",")]
mass = np.array(mass)
print(len(mass))

212


In [38]:
main_csv_path = os.path.join(survive_data_folder, "main_halo_com.csv")

try:
    main_csv_df= pd.read_csv(main_csv_path, dtype= {"snapshot":str})
except Exception as e:
    print(f"File loading error, {e}")
SNAPSHOT_TO_ANALYZE=4096
a=f"{SNAPSHOT_TO_ANALYZE:06d}"
print(a)
print(type(a))
main_csv_snap=main_csv_df[main_csv_df["snapshot"]==a]
if not main_csv_snap.empty:
    x_center=main_csv_snap["0"].iloc[0]
    y_center=main_csv_snap["1"].iloc[0]
    z_center=main_csv_snap["2"].iloc[0]
    print(x_center,y_center,z_center)
else:
    print("Data not found")

004096
<class 'str'>
1877.44745057552 -4441.044233960943 1478.1246825739918
